In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!nvidia-smi

Tue Aug  3 07:02:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 14.1 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 636 kB 43.0 MB/s 
     |████████████████████████████████| 3.3 MB 44.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install pytorch-lightning

     |████████████████████████████████| 913 kB 13.1 MB/s 
     |████████████████████████████████| 118 kB 47.6 MB/s 
     |████████████████████████████████| 10.6 MB 141 kB/s 
     |████████████████████████████████| 829 kB 38.5 MB/s 
     |████████████████████████████████| 234 kB 47.1 MB/s 
     |████████████████████████████████| 1.3 MB 40.3 MB/s 
     |████████████████████████████████| 294 kB 48.7 MB/s 
     |████████████████████████████████| 142 kB 32.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=e1544c0e654c89effe7f75d46c5196405a572e1f1233e39711934c607913bff4
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: future
    Found existing installation: future 0.16.0
 

In [5]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import auroc

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

In [6]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Biocreative/Biocreative/litcovid_dataset.csv')

In [8]:
df.head(n=20)

,pmid,journal,title,abstract,keywords,pub_type,authors,doi,label,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,32519164,J Thromb Thrombolysis,Potential role for tissue factor in the pathog...,"In December 2019, a new and highly contagious ...",covid-19;il-6;sars-cov-2;tnf-alpha;thrombosis;...,Journal Article;Review,"Bautista-Vargas, Mario;Bonilla-Abadia, Fabio;C...",10.1007/s11239-020-02172-x,Treatment;Mechanism,0,0,0,1,0,0,1
1,32691006,J Tradit Complement Med,Dietary therapy and herbal medicine for COVID-...,"A novel coronavirus disease (COVID-19), transm...",covid-19;coronavirus;dietary therapy;herbal me...,Journal Article;Review,"Panyod, Suraphan;Ho, Chi-Tang;Sheen, Lee-Yan",10.1016/j.jtcme.2020.05.004,Treatment;Prevention,0,0,0,0,1,0,1
2,32858315,J Affect Disord,First report of manic-like symptoms in a COVID...,"BACKGROUND: In December 2019, the novel corona...",cerebrospinal fluid;igg;manic-like symptoms;sa...,Case Reports;Journal Article,"Lu, Shaojia;Wei, Ning;Jiang, Jiajun;Wu, Lingli...",10.1016/j.jad.2020.08.031,Case Report,1,0,0,0,0,0,0
3,32985329,J Dent Res,Epidemiological Investigation of OHCWs with CO...,During the coronavirus disease 2019 (COVID-19)...,dental education;dental public health;infectio...,"Journal Article;Research Support, Non-U.S. Gov't","Meng, L;Ma, B;Cheng, Y;Bian, Z",10.1177/0022034520962087,Prevention,0,0,0,0,1,0,0
4,32812051,J Antimicrob Chemother,The impact of sofosbuvir/daclatasvir or ribavi...,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,NaN,Journal Article;Randomized Controlled Trial;Re...,"Eslami, Gholamali;Mousaviasl, Sajedeh;Radmanes...",10.1093/jac/dkaa331,Treatment,0,0,0,0,0,0,1
5,32926870,Chest,The Coronavirus Disease 2019 Pandemic's Effect...,BACKGROUND: The coronavirus disease 2019 (COVI...,covid-19;burnout;critical care;emotional distr...,Journal Article,"Wahlster, Sarah;Sharma, Monisha;Lewis, Ariane ...",10.1016/j.chest.2020.09.070,Prevention,0,0,0,0,1,0,0
6,32611634,Neurology,Accelerated launch of video visits in ambulato...,The SARS-CoV-2 (COVID-19) pandemic has rapidly...,NaN,"Journal Article;Research Support, Non-U.S. Gov't","Yang, Laurice;Brown-Johnson, Cati G;Miller-Kuh...",10.1212/WNL.0000000000010015,Prevention,0,0,0,0,1,0,0
7,32207676,Disaster Med Public Health Prep,COVID-19 Epidemic in the Middle Province of No...,The novel coronavirus (COVID-19) began in Chin...,covid-19;emergency;public health;strategy,Journal Article,"Gagliano, Annalisa;Villani, Pier Giorgio;Co', ...",10.1017/dmp.2020.51,Prevention,0,0,0,0,1,0,0
8,32563601,Ann Emerg Med,Cohort of Four Thousand Four Hundred Four Pers...,Study objective: Most coronavirus disease 2019...,NaN,Journal Article,"Singer, Adam J;Morley, Eric J;Meyers, Kristen;...",10.1016/j.annemergmed.2020.05.011,Diagnosis;Treatment,0,1,0,0,0,0,1
9,32778470,J Prosthet Dent,Coronavirus disease 2019 (COVID-19): Experienc...,STATEMENT OF PROBLEM: The novel Coronavirus Di...,NaN,Journal Article,"Sa, Yue;Lin, Wei-Shao;Morton, Dean;Huang, Cui",10.1016/j.prosdent.2020.06.004,Prevention,0,0,0,0,1,0,0


In [9]:
df.shape

(24960, 16)

In [10]:
df.columns

Index(['pmid', 'journal', 'title', 'abstract', 'keywords', 'pub_type',
       'authors', 'doi', 'label', 'Case Report', 'Diagnosis',
       'Epidemic Forecasting', 'Mechanism', 'Prevention', 'Transmission',
       'Treatment'],
      dtype='object')

In [11]:
# removing unnecessary columns
Data = df.drop(['pmid', 'journal', 'title', 'keywords', 'pub_type', 'authors', 'doi', 'label'], axis=1)
Data.columns

Index(['abstract', 'Case Report', 'Diagnosis', 'Epidemic Forecasting',
       'Mechanism', 'Prevention', 'Transmission', 'Treatment'],
      dtype='object')

In [12]:
Data.head(n=20)

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,"In December 2019, a new and highly contagious ...",0,0,0,1,0,0,1
1,"A novel coronavirus disease (COVID-19), transm...",0,0,0,0,1,0,1
2,"BACKGROUND: In December 2019, the novel corona...",1,0,0,0,0,0,0
3,During the coronavirus disease 2019 (COVID-19)...,0,0,0,0,1,0,0
4,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,0,0,0,0,0,0,1
5,BACKGROUND: The coronavirus disease 2019 (COVI...,0,0,0,0,1,0,0
6,The SARS-CoV-2 (COVID-19) pandemic has rapidly...,0,0,0,0,1,0,0
7,The novel coronavirus (COVID-19) began in Chin...,0,0,0,0,1,0,0
8,Study objective: Most coronavirus disease 2019...,0,1,0,0,0,0,1
9,STATEMENT OF PROBLEM: The novel Coronavirus Di...,0,0,0,0,1,0,0


In [13]:
# shape of DataFrame to determine the train_test_split
Data.shape

(24960, 8)

In [14]:
# splitting the data
train_df, val_df = train_test_split(Data, test_size=0.05)

In [15]:
train_df.shape, val_df.shape

((23712, 8), (1248, 8))

In [16]:
# constant list to represent all the classes
LABEL_COLUMNS = list(Data.columns)[1:]
LABEL_COLUMNS

['Case Report',
 'Diagnosis',
 'Epidemic Forecasting',
 'Mechanism',
 'Prevention',
 'Transmission',
 'Treatment']

In [17]:
# share of classes in train dataframe
train_df[LABEL_COLUMNS].sum()

Case Report              1958
Diagnosis                5873
Epidemic Forecasting      619
Mechanism                4221
Prevention              10549
Transmission             1031
Treatment                8281
dtype: int64

In [18]:
sample_row = train_df.iloc[0]
sample_abstract = sample_row.abstract
sample_labels = sample_row[LABEL_COLUMNS]

print(sample_abstract)
print()
print(sample_labels.to_dict())

After the emergence of the novel 2019 coronavirus disease in P. R. China, this highly contagious disease has been currently spread out to almost all countries, worldwide. Novel 2019 coronavirus disease, Middle East respiratory syndrome, and severe acute respiratory syndrome are reported to cause a higher risk for severe infections in patients with chronic comorbidities, such as hypertension and diabetes. These severe infections can contribute to higher rates of morbidity and mortality in these patients. In the present review, we discussed the role and underlying mechanisms of the two most common chronic diseases, type-2 diabetes mellitus and hypertension, in clinical manifestations and disease severity of novel 2019 coronavirus disease, Middle East respiratory syndrome and severe acute respiratory syndrome, with the hope to provide evidence for better decision-making in the treatment of this vulnerable population.

{'Case Report': 0, 'Diagnosis': 1, 'Epidemic Forecasting': 0, 'Mechanis

In [19]:
BERT_MODEL_NAME = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [20]:
encoding = tokenizer.encode_plus(
    sample_abstract,
    add_special_tokens=True,
    max_length=512,
    return_token_type_ids=False,
    padding="max_length",
    return_attention_mask=True,
    return_tensors="pt"
)

In [21]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [22]:
encoding['input_ids'].shape, encoding['attention_mask'].shape

(torch.Size([1, 512]), torch.Size([1, 512]))

In [23]:
encoding['input_ids'].squeeze()[:20]

tensor([  101,  1258,  1103, 15351,  1104,  1103,  2281, 10351,  1884, 15789,
        27608,  3653,  1107,   153,   119,   155,   119,  1975,   117,  1142])

In [24]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze())[:20])

['[CLS]', 'After', 'the', 'emergence', 'of', 'the', 'novel', '2019', 'co', '##rona', '##virus', 'disease', 'in', 'P', '.', 'R', '.', 'China', ',', 'this']


In [25]:
N_EPOCHS = 10
BATCH_SIZE = 12 
MAX_TOKEN_COUNT = 512

In [26]:
class TopicAnnotationDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_token_len: int = 128):

    self.data = data
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def __len__(self):

    return len(self.data)

  def __getitem__(self, index: int):

    data_row = self.data.iloc[index]

    abstract_article = data_row.abstract
    labels = data_row[LABEL_COLUMNS]

    encoding = self.tokenizer.encode_plus(
        abstract_article,
        add_special_tokens=True,
        max_length=self.max_token_len,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )

    return dict(
        abstract_article = abstract_article,
        input_ids = encoding["input_ids"].flatten(),
        attention_mask = encoding["attention_mask"].flatten(),
        labels = torch.FloatTensor(labels)
    )

In [27]:
# intializing train_dataset using above mentioned class and passing in train datfarame and BertTokenizer
train_dataset = TopicAnnotationDataset(
    train_df,
    tokenizer,
    max_token_len=MAX_TOKEN_COUNT
)

In [28]:
# try running one example
sample_item = next(iter(train_dataset))
sample_item.keys()

dict_keys(['abstract_article', 'input_ids', 'attention_mask', 'labels'])

In [29]:
# creating bert model using the huggingface transformer library
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
sample_batch = next(iter(DataLoader(train_dataset, batch_size=8, num_workers=2)))
sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape

(torch.Size([8, 512]), torch.Size([8, 512]))

In [31]:
# prediction = bert_model(sample_item["input_ids"].unsqueeze(dim=0), sample_item["attention_mask"].unsqueeze(dim=0))
output = bert_model(sample_batch["input_ids"], sample_batch["attention_mask"])

In [32]:
# prediction.last_hidden_state.shape, prediction.pooler_output.shape
output.last_hidden_state.shape, output.pooler_output.shape

(torch.Size([8, 512, 768]), torch.Size([8, 768]))

In [33]:
bert_model.config.hidden_size

768

In [34]:
class TopicAnnotationDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
    
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.max_token_len = max_token_len

  def setup(self):

    self.train_dataset = TopicAnnotationDataset(
        self.train_df,
        self.tokenizer,
        self.max_token_len
    )

    self.test_dataset = TopicAnnotationDataset(
        self.test_df,
        self.tokenizer,
        self.max_token_len
    )

    def train_dataloader(self):
      return DataLoader(
          self.train_dataset,
          batch_size = self.batch_size,
          shuffle = True,
          num_workers = 2
      )

    def val_dataloader(self):
      return DataLoader(
          self.test_dataset, 
          batch_size = 1, 
          num_workers = 4
      )

    def test_dataloader(self):
      return DataLoader(
          self.test_dataset,
          batch_size = 1,
          num_workers = 4
      )

In [35]:
data_module = TopicAnnotationDataModule(
    train_df,
    val_df,
    tokenizer,
    batch_size=BATCH_SIZE,
    max_token_len=MAX_TOKEN_COUNT    
)
data_module.setup()

## Modeling

### Evaluation

In [36]:
 class TopicAnnotationTagger(pl.LightningModule):

  def __init__(self, n_classes: int, steps_per_epoch=None, n_epochs=None):
     
    super().__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.steps_per_epoch = steps_per_epoch
    self.n_epochs = n_epochs
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):

    output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, output = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss,   "predictions": output, "labels": labels}

  def validation_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, output = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, output = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def training_epoch_end(self, outputs):

    labels = []
    predictions = []

    for output in outputs:
      for out_labels in output["labels"].detach().cpu():
        labels.append(out_labels)

      for out_predictions in output["predictions"].detach().cpu():
        predictions.append(out_predictions)

    labels = torch.stack(labels)
    predictions = torch.stack(predictions)
    
    for i, name in enumerate(LABEL_COLUMNS):
      roc_score = auroc(predictions[:, i], labels[:, i])
      self.logger.experiment.add_scalar(f"{name}_roc_auc/TRAIN", roc_score, self.current_epoch)

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=2e-5)
    
    warmup_steps = self.steps_per_epoch // 3
    total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps

    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        warmup_steps,
        total_steps
    )
  
    return [optimizer], [scheduler]

In [37]:
class TopicAnnotationTagger(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    
    super().__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.BCELoss()
  
  def forward(self, input_ids, attention_mask, labels=None):
    
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    
    return loss, output
  
  def training_step(self, batch, batch_idx):
    
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}
  
  def configure_optimizers(self):
    
    optimizer = AdamW(self.parameters(), lr=2e-5)
    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )
    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )
    
  def validation_step(self, batch, batch_idx):
   
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  
  def test_step(self, batch, batch_idx):
    
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss
  
  def training_epoch_end(self, outputs):
    
    labels = []
    predictions = []
    for output in outputs:
      for out_labels in output["labels"].detach().cpu():
        labels.append(out_labels)
      for out_predictions in output["predictions"].detach().cpu():
        predictions.append(out_predictions)
    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)
    for i, name in enumerate(LABEL_COLUMNS):
      class_roc_auc = auroc(predictions[:, i], labels[:, i])
      self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)

In [38]:
steps_per_epoch = len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [39]:
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(3952, 19760)

In [40]:
# model = TopicAnnotationTagger(
#     n_classes=len(LABEL_COLUMNS),
#     steps_per_epoch=len(train_df) // BATCH_SIZE, 
#     n_epochs=N_EPOCHS
# )

model = TopicAnnotationTagger(
    n_classes=len(LABEL_COLUMNS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
# _, predictions = model(
#     sample_item["input_ids"].unsqueeze(dim=0),
#     sample_item["attention_mask"].unsqueeze(dim=0)
# )
_, predictions = model(
    sample_batch["input_ids"],
    sample_batch["attention_mask"]
)

In [42]:
predictions

tensor([[0.2990, 0.4390, 0.4318, 0.7266, 0.4144, 0.4654, 0.4129],
        [0.3099, 0.4373, 0.4274, 0.7229, 0.4020, 0.4653, 0.4143],
        [0.3383, 0.3550, 0.4300, 0.6906, 0.4606, 0.4294, 0.4622],
        [0.3196, 0.4308, 0.4186, 0.7267, 0.4190, 0.4383, 0.3901],
        [0.3062, 0.4445, 0.4282, 0.7349, 0.4157, 0.4456, 0.4114],
        [0.3079, 0.4429, 0.4200, 0.7324, 0.4017, 0.4539, 0.4061],
        [0.3040, 0.4413, 0.4348, 0.7205, 0.4082, 0.4647, 0.4138],
        [0.3063, 0.4061, 0.4283, 0.7170, 0.4151, 0.4555, 0.4201]],
       grad_fn=<SigmoidBackward>)

In [43]:
trainer = pl.Trainer(
  max_epochs=N_EPOCHS,
  gpus=1,
  progress_bar_refresh_rate=30
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [44]:
# trainer.fit(model, data_module)